In [1]:
import os
from typing import List, Tuple, Dict

import joblib
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import HistGradientBoostingRegressor

In [3]:
# 固定随机种子，保证可复现
RANDOM_STATE = 42

# 数据路径与目标列
CSV_PATH = "wave3ndata.csv"     # <-- 改为你的CSV路径
TARGET_COL = "adlab_c"

# 特征筛选阈值（与前面方案一致）
CORR_TH = 0.05        # 与目标的绝对相关下限
REDUNDANCY_TH = 0.90  # 特征-特征冗余阈值
TOP_N = 30            # Feature-Reduced 的 Top-N

# 输出目录
OUTDIR = "models_exp_HGB"
os.makedirs(OUTDIR, exist_ok=True)

In [4]:
#工具函数集
def eval_report(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    """统一评估：回归指标 + 业务友好指标"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    y_round = np.clip(np.rint(y_pred), 0, 6).astype(int)
    acc = accuracy_score(y_true, y_round)
    within1 = float(np.mean(np.abs(y_true - y_round) <= 1))
    return dict(MAE=mae, RMSE=rmse, R2=r2, Acc_rounded=acc, Within1=within1)

def split_numeric(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    """仅保留数值特征列，剔除目标列"""
    return df[[c for c in df.columns if c != target_col and pd.api.types.is_numeric_dtype(df[c])]].copy()

def detect_bin_con_cols(X: pd.DataFrame) -> Tuple[List[str], List[str]]:
    """粗分二值/连续列：唯一值是否子集于{0,1}"""
    bin_cols, con_cols = [], []
    for c in X.columns:
        vals = pd.unique(X[c].dropna())
        if len(vals) and set(np.unique(vals)).issubset({0,1}):
            bin_cols.append(c)
        else:
            con_cols.append(c)
    return bin_cols, con_cols

def fit_impute_stats(X_train: pd.DataFrame, bin_cols: List[str], con_cols: List[str]) -> dict:
    """训练集上拟合缺失填充值：二值=众数；连续=中位数"""
    stats = {"binary_modes": {}, "continuous_medians": {}}
    for c in bin_cols:
        s = X_train[c]
        stats["binary_modes"][c] = float(s.mode().iloc[0]) if not s.dropna().empty else 0.0
    for c in con_cols:
        s = X_train[c]
        stats["continuous_medians"][c] = float(s.median()) if not s.dropna().empty else 0.0
    return stats

def apply_impute_inplace(X: pd.DataFrame, stats: dict) -> None:
    """按训练统计值原地填充缺失"""
    for c, v in stats["binary_modes"].items():
        if c in X.columns:
            X[c] = X[c].fillna(v)
    for c, v in stats["continuous_medians"].items():
        if c in X.columns:
            X[c] = X[c].fillna(v)

def screen_features_on_train(
    X_train: pd.DataFrame, y_train: np.ndarray,
    corr_th: float, redundancy_th: float, top_n: int
) -> List[str]:
    """三阶段筛选 + HGB 重要度选 Top-N（仅用训练集，以避免泄漏）"""
    # 1) 低方差
    vt = VarianceThreshold(0.0)
    X1 = X_train.loc[:, vt.fit(X_train).get_support()]
    # 2) 与目标相关
    kept = []
    for c in X1.columns:
        try:
            r = abs(pearsonr(X1[c], y_train)[0])
        except Exception:
            r = 0.0
        if r > corr_th:
            kept.append(c)
    X2 = X1[kept] if kept else X1
    # 3) 冗余
    if X2.shape[1] > 1:
        cm = X2.corr().abs()
        upper = cm.where(np.triu(np.ones(cm.shape), k=1).astype(bool))
        drop = [col for col in upper.columns if any(upper[col] > redundancy_th)]
        X3 = X2.drop(columns=drop) if drop else X2
    else:
        X3 = X2
    # 4) Top-N
    if X3.shape[1] > 0:
        hgb = HistGradientBoostingRegressor(
            learning_rate=0.05, max_leaf_nodes=31, min_samples_leaf=20, random_state=RANDOM_STATE
        )
        hgb.fit(X3, y_train)
        imps = pd.Series(hgb.feature_importances_, index=X3.columns).sort_values(ascending=False)
        selected = imps.head(min(top_n, X3.shape[1])).index.tolist()
    else:
        selected = []
    return selected


In [5]:
# 读取CSV
df = pd.read_csv(CSV_PATH)
assert TARGET_COL in df.columns, f"CSV 中缺少目标列 {TARGET_COL}"

# 目标列裁剪到[0,6]并取整（避免脏标注；保证整数等级）
df = df.dropna(subset=[TARGET_COL]).copy()
df[TARGET_COL] = df[TARGET_COL].astype(float).round().clip(0,6).astype(int)

# 数值特征与目标
X_all = split_numeric(df, TARGET_COL)
y_all = df[TARGET_COL].values

# 分层切分（保持0~6各等级比例）
X_tr, X_te, y_tr, y_te = train_test_split(
    X_all, y_all, test_size=0.2, random_state=RANDOM_STATE, stratify=y_all
)

# 训练集上拟合缺失填充统计，并对Train/Test应用（防止信息泄漏）
bin_cols, con_cols = detect_bin_con_cols(X_tr)
impute_stats = fit_impute_stats(X_tr, bin_cols, con_cols)
apply_impute_inplace(X_tr, impute_stats)
apply_impute_inplace(X_te, impute_stats)

X_tr.shape, X_te.shape, len(bin_cols), len(con_cols)


((5204, 63), (1302, 63), 47, 16)

In [6]:
# 纯HGB 
baseline_hgb = HistGradientBoostingRegressor(
    learning_rate=0.05, max_leaf_nodes=31, min_samples_leaf=20, random_state=RANDOM_STATE
)
baseline_hgb.fit(X_tr, y_tr)
pred_base = baseline_hgb.predict(X_te)
metrics_base = eval_report(y_te, pred_base)
metrics_base


c:\Users\11466\Anaconda\envs\Yarn_prodiction\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'MAE': 0.5755583554442748,
 'RMSE': 0.9020973959150392,
 'R2': 0.5691368284614929,
 'Acc_rounded': 0.6382488479262672,
 'Within1': 0.9001536098310292}

In [7]:

#PSO+HGB
# ----- PSO 配置（可调） -----
SWARM_SIZE = 12        # 粒子数量（可增大提升稳定性）
N_ITERS = 15           # 迭代轮数（可增大提升精度）
W, C1, C2 = 0.6, 1.5, 1.5  # 惯性权重、个体/群体学习因子

# 参数边界
BOUNDS = {
    "learning_rate": (0.01, 0.30),
    "max_leaf_nodes": (15, 63),
    "min_samples_leaf": (5, 50),
    "l2_regularization": (0.0, 1.0),
    "max_iter": (100, 500),
}
PARAM_KEYS = list(BOUNDS.keys())

rng = np.random.default_rng(RANDOM_STATE)

def sample_position():
    """在边界内随机初始化一个粒子位置向量"""
    pos = []
    for k in PARAM_KEYS:
        lo, hi = BOUNDS[k]
        pos.append(rng.uniform(lo, hi))
    return np.array(pos, dtype=float)

def clip_position(pos):
    """将位置限制在边界内"""
    clipped = pos.copy()
    for i, k in enumerate(PARAM_KEYS):
        lo, hi = BOUNDS[k]
        clipped[i] = np.clip(clipped[i], lo, hi)
    return clipped

def position_to_params(pos):
    """将位置向量映射到HGB参数字典（需要取整的取整）"""
    d = dict(zip(PARAM_KEYS, pos.tolist()))
    d["max_leaf_nodes"] = int(round(d["max_leaf_nodes"]))
    d["min_samples_leaf"] = int(round(d["min_samples_leaf"]))
    d["max_iter"] = int(round(d["max_iter"]))
    return d

def cv_mae_for_params(params, X, y, n_splits=3):
    """给定参数在训练集上做K折分层CV，返回平均MAE（越小越好）"""
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    maes = []
    for tr_idx, va_idx in skf.split(X, y):
        Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
        ytr, yva = y[tr_idx], y[va_idx]
        # 直接使用训练阶段的填充值与完整特征（不做特征筛选）以控制变量
        model = HistGradientBoostingRegressor(
            learning_rate=params["learning_rate"],
            max_leaf_nodes=params["max_leaf_nodes"],
            min_samples_leaf=params["min_samples_leaf"],
            l2_regularization=params["l2_regularization"],
            max_iter=params["max_iter"],
            random_state=RANDOM_STATE
        )
        model.fit(Xtr, ytr)
        pred = model.predict(Xva)
        maes.append(mean_absolute_error(yva, pred))
    return float(np.mean(maes))

# 初始化粒子群
positions = np.stack([sample_position() for _ in range(SWARM_SIZE)], axis=0)
velocities = rng.normal(loc=0.0, scale=0.1, size=positions.shape)

# 个体/全局最优
pbest_pos = positions.copy()
pbest_val = np.array([
    cv_mae_for_params(position_to_params(p), X_tr, y_tr) for p in positions
], dtype=float)
gbest_idx = int(np.argmin(pbest_val))
gbest_pos = pbest_pos[gbest_idx].copy()
gbest_val = pbest_val[gbest_idx].item()

history = [gbest_val]

# 迭代优化
for it in range(N_ITERS):
    for i in range(SWARM_SIZE):
        r1, r2 = rng.random(len(PARAM_KEYS)), rng.random(len(PARAM_KEYS))
        velocities[i] = (
            W * velocities[i]
            + C1 * r1 * (pbest_pos[i] - positions[i])
            + C2 * r2 * (gbest_pos - positions[i])
        )
        positions[i] = clip_position(positions[i] + velocities[i])
        # 评估
        val = cv_mae_for_params(position_to_params(positions[i]), X_tr, y_tr)
        if val < pbest_val[i]:
            pbest_val[i] = val
            pbest_pos[i] = positions[i].copy()
            if val < gbest_val:
                gbest_val = val
                gbest_pos = positions[i].copy()
    history.append(gbest_val)

best_params_pso = position_to_params(gbest_pos)
best_params_pso, history[-1]


({'learning_rate': 0.012512501432968153,
  'max_leaf_nodes': 15,
  'min_samples_leaf': 50,
  'l2_regularization': 1.0,
  'max_iter': 500},
 0.5968994442463676)

In [8]:
hgb_pso = HistGradientBoostingRegressor(
    **best_params_pso, random_state=RANDOM_STATE
)
hgb_pso.fit(X_tr, y_tr)
pred_pso = hgb_pso.predict(X_te)
metrics_pso = eval_report(y_te, pred_pso)
metrics_pso, best_params_pso


c:\Users\11466\Anaconda\envs\Yarn_prodiction\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


({'MAE': 0.5665884240643839,
  'RMSE': 0.8897181010294642,
  'R2': 0.5808809817817147,
  'Acc_rounded': 0.6397849462365591,
  'Within1': 0.9024577572964669},
 {'learning_rate': 0.012512501432968153,
  'max_leaf_nodes': 15,
  'min_samples_leaf': 50,
  'l2_regularization': 1.0,
  'max_iter': 500})

In [9]:
# Weight+HGB

# 计算训练集各等级频率
unique, counts = np.unique(y_tr, return_counts=True)
freq = dict(zip(unique, counts))
inv_freq = {k: 1.0 / v for k, v in freq.items()}

# 将逆频率缩放到均值=1，避免权重过大影响收敛
w = np.array([inv_freq[y] for y in y_tr], dtype=float)
w = w * (len(w) / np.sum(w))  # scale so mean ~ 1

# 使用与基线一致的模型结构，仅加入 sample_weight
hgb_w = HistGradientBoostingRegressor(
    learning_rate=0.05, max_leaf_nodes=31, min_samples_leaf=20, random_state=RANDOM_STATE
)
hgb_w.fit(X_tr, y_tr, sample_weight=w)
pred_w = hgb_w.predict(X_te)
metrics_w = eval_report(y_te, pred_w)
metrics_w


c:\Users\11466\Anaconda\envs\Yarn_prodiction\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'MAE': 0.9139693804046484,
 'RMSE': 1.1577039779696394,
 'R2': 0.29037719028644315,
 'Acc_rounded': 0.34715821812596004,
 'Within1': 0.8233486943164362}

In [11]:
results = pd.DataFrame([
    dict(Exp="Baseline-HGB", **metrics_base),
    dict(Exp="PSO-HGB", **metrics_pso),
    dict(Exp="Weighted-HGB", **metrics_w),
])

# 四舍五入显示
display(results.round(4))

# 可选：保存结果
results.to_csv(os.path.join(OUTDIR, "exp_results_summary.csv"), index=False)


,Exp,MAE,RMSE,R2,Acc_rounded,Within1
0,Baseline-HGB,0.5756,0.9021,0.5691,0.6382,0.9002
1,PSO-HGB,0.5666,0.8897,0.5809,0.6398,0.9025
2,Weighted-HGB,0.9140,1.1577,0.2904,0.3472,0.8233


In [ ]:
import matplotlib.pyplot as plt

metrics_to_plot = ["MAE","RMSE","R2","Acc_rounded","Within1"]
fig, ax = plt.subplots(figsize=(8,4))
results_melt = results.melt(id_vars="Exp", value_vars=metrics_to_plot, var_name="Metric", value_name="Value")
for m in metrics_to_plot:
    subset = results_melt[results_melt["Metric"]==m]
    ax.plot(subset["Exp"], subset["Value"], marker="o", label=m)
ax.set_title("Experiment Comparison (HGB Variants)")
ax.set_xticklabels(results["Exp"], rotation=15)
ax.grid(True, linestyle="--", alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()
